In [1]:
base_df = 'kaggle/'
stock_df = 'kaggle/stocks/'
etfs_df = 'kaggle/etfs/'

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
valid_data = pd.read_csv(stock_df+"DFNS.csv")

In [4]:
len(valid_data)

6

In [5]:
valid_data['Date'].unique

<bound method Series.unique of 0    2020-03-25
1    2020-03-26
2    2020-03-27
3    2020-03-30
4    2020-03-31
5    2020-04-01
Name: Date, dtype: object>

In [6]:

def datasetConstruct(arq):
    try:
        a_data = pd.read_csv(os.path.join(stock_df,arq))
        a_data.dropna()
            
        a_data['Date'] = pd.to_datetime(a_data['Date'])
        a_data['Day'] = a_data['Date'].dt.day 
        a_data['Month'] = a_data['Date'].dt.month
        a_data['Year'] = a_data['Date'].dt.year
        a_data['Week'] = a_data['Date'].dt.isocalendar().week
        a_data['day_name'] = a_data['Date'].dt.day_name()
        
        week_data = a_data.groupby(['Year','Week'])
        contagem = a_data[['Year','Week']].value_counts()
        valores_maior_ou_igual_n = contagem[contagem >= 5].index
        week_data = a_data[a_data.set_index(['Year','Week']).index.isin(valores_maior_ou_igual_n)]
        week_data = week_data.drop_duplicates(subset=['Year', 'Week', 'day_name'], keep='first')
        data = week_data.pivot(index=['Year','Week'], columns=['day_name'], values='Close').reset_index()
        data['Target'] = np.where(data['Friday'] > data['Thursday'], 1, 0)
        return data
    except:
        print(arq)
        a_data = pd.DataFrame()
        return a_data

    
    





In [7]:
data = pd.DataFrame()
lista = os.listdir(stock_df)
# arq = lista[0]
for arq in lista:
    if '.csv' in arq:
        dataConstruct = datasetConstruct(arq=arq)
        data = pd.concat([data,dataConstruct])
    
data = data.reset_index(drop=True)
data

DFNS.csv
RILYP.csv
GHIV.csv
SCA.csv
METX.csv


day_name,Year,Week,Friday,Monday,Thursday,Tuesday,Wednesday,Target,Sunday,Saturday
0,2015,45,4.650,4.600000,4.650000,4.650000,4.650000,0,NaN,NaN
1,2015,46,4.500,4.650000,4.500000,4.650000,4.650000,0,NaN,NaN
2,2015,47,4.650,4.500000,4.650000,4.300000,4.300000,0,NaN,NaN
3,2015,49,4.750,4.650000,4.750000,4.650000,4.900000,0,NaN,NaN
4,2015,50,4.750,4.750000,4.750000,4.750000,4.750000,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
4181582,2020,9,48.590,42.703999,46.835999,44.540001,44.988998,1,NaN,NaN
4181583,2020,10,52.243,45.525002,50.419998,48.305000,47.493999,1,NaN,NaN
4181584,2020,11,55.090,59.023998,64.584000,54.890999,57.985001,0,NaN,NaN
4181585,2020,12,63.270,62.830002,61.632000,59.175999,62.757999,1,NaN,NaN


In [8]:
data = data[['Monday','Tuesday','Wednesday','Thursday','Target']]
data = data.dropna()
data

day_name,Monday,Tuesday,Wednesday,Thursday,Target
0,4.600000,4.650000,4.650000,4.650000,0
1,4.650000,4.650000,4.650000,4.500000,0
2,4.500000,4.300000,4.300000,4.650000,0
3,4.650000,4.650000,4.900000,4.750000,0
4,4.750000,4.750000,4.750000,4.750000,0
...,...,...,...,...,...
4181582,42.703999,44.540001,44.988998,46.835999,1
4181583,45.525002,48.305000,47.493999,50.419998,1
4181584,59.023998,54.890999,57.985001,64.584000,0
4181585,62.830002,59.175999,62.757999,61.632000,1


In [9]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
day_name,,,,,,,,
Monday,4181499.0,3.106037e+06,5.327466e+08,0.0,6.690000,14.660000,29.450001,1.732752e+11
Tuesday,4181499.0,3.089941e+06,5.286055e+08,0.0,6.694445,14.666667,29.468750,1.704780e+11
Wednesday,4181499.0,3.092335e+06,5.288136e+08,0.0,6.700000,14.666667,29.480000,1.587600e+11
Thursday,4181499.0,3.084002e+06,5.271531e+08,0.0,6.690000,14.660000,29.459999,1.490832e+11
Target,4181499.0,4.458114e-01,4.970550e-01,0.0,0.000000,0.000000,1.000000,1.000000e+00


In [10]:
data.sample(frac=1, random_state=42).reset_index(drop=True)

day_name,Monday,Tuesday,Wednesday,Thursday,Target
0,37.779999,36.790001,35.910000,36.200001,0
1,14.520000,14.200000,14.250000,14.300000,0
2,30.190001,29.730000,28.590000,28.320000,1
3,75.500000,74.019997,71.900002,71.269997,0
4,16.615000,16.495001,16.150000,15.950000,1
...,...,...,...,...,...
4181494,17.120001,17.379999,17.629999,17.049999,1
4181495,31.400000,31.200001,31.600000,31.629999,1
4181496,17.530001,17.000000,17.160000,17.240000,1
4181497,21.070000,20.990000,20.520000,20.090000,0


In [11]:
X,y = data[['Monday','Tuesday','Wednesday','Thursday']],data['Target']

In [12]:
X.isnull().sum()

day_name
Monday       0
Tuesday      0
Wednesday    0
Thursday     0
dtype: int64

In [13]:
y.isnull().sum()

np.int64(0)

In [14]:
from sklearn import preprocessing

X = np.asarray(X).astype('float32')
# X = np.nan_to_num(X,nan=0)

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(x_scaled)

In [15]:
X.isnull().sum()

0    0
1    0
2    0
3    0
dtype: int64

In [16]:
y = np.asarray(y).astype('int')
y

array([0, 0, 0, ..., 0, 1, 1])

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

print('Shape de x_train', x_train.shape)
print('Shape de x_val', x_val.shape)
print('Shape de x_test', x_test.shape)

print('Shape de y_train', y_train.shape)
print('Shape de y_val', y_val.shape)
print('Shape de y_test', y_test.shape)
x_train = np.expand_dims(x_train, axis=-1)
print(x_train.shape) 

Shape de x_train (2341639, 4)
Shape de x_val (1003560, 4)
Shape de x_test (836300, 4)
Shape de y_train (2341639,)
Shape de y_val (1003560,)
Shape de y_test (836300,)
(2341639, 4, 1)


In [24]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Input, BatchNormalization, LeakyReLU, Conv1D, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam, RMSprop, Adadelta, Adamax
from tensorflow.keras.callbacks import EarlyStopping

Adam = Adam(
    learning_rate=0.0001
)

model = Sequential([
    Input(shape=(x_train.shape[1], x_train.shape[2])),
    Conv1D(128, kernel_size=1, activation='selu',padding="same"),
    Conv1D(64, kernel_size=3, activation='selu',padding="same"),
    Conv1D(32, kernel_size=3, activation='selu',padding="same"),
    Conv1D(16, kernel_size=3, activation='selu',padding="same"),
    Conv1D(8, kernel_size=3, activation='selu',padding="same"),
    Conv1D(4, kernel_size=3, activation='selu',padding="same"),
    Conv1D(2, kernel_size=3, activation='selu',padding="same"),
    
    # Use GlobalAveragePooling1D ou Flatten para transformar em 2D
    GlobalAveragePooling1D(),
    
    # Agora, adicione as camadas Dense
    Dense(128, activation='tanh'),
    Dense(64, activation='relu'),
    Dense(32, activation='tanh'),
    Dense(16, activation='selu'),
    Dense(8, activation='tanh'),
    Dense(4, activation='selu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer=RMSprop(learning_rate=0.00001), metrics=['accuracy'])

In [25]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 4, 128)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 4, 64)          │        24,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 4, 32)          │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 4, 16)          │         1,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_18 (Conv1D)              │ (None, 4, 8)           │           392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_19 (Conv1D)              │ (None, 4, 4)           │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 4, 2)           │            26 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_2      │ (None, 2)              │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 4)              │            36 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │             5 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 44,567 (174.09 KB)

 Trainable params: 44,567 (174.09 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
batch_size = 256
epochs = 300
callback = EarlyStopping(
    monitor = 'accuracy',
    min_delta = 0.01,
    patience = 10,
    verbose = 1
)
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                   callbacks = callback)

Epoch 1/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - accuracy: 0.5543 - loss: 0.6875 - val_accuracy: 0.5543 - val_loss: 0.6873
Epoch 2/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - accuracy: 0.5542 - loss: 0.6873 - val_accuracy: 0.5543 - val_loss: 0.6872
Epoch 3/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.5540 - loss: 0.6873 - val_accuracy: 0.5543 - val_loss: 0.6873
Epoch 4/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.5541 - loss: 0.6873 - val_accuracy: 0.5543 - val_loss: 0.6872
Epoch 5/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.5543 - loss: 0.6873 - val_accuracy: 0.5543 - val_loss: 0.6873
Epoch 6/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 37s 4ms/step - accuracy: 0.5544 - loss: 0.6872 - val_accuracy: 0.5543 - val_loss: 0.6872
Epoch 7/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.5540 - loss: 0.6873 - val_accuracy: 0.5543 - val_loss: 0.6872
Epoch 8/300
9148/9148 ━━━━━━━━━━━━━━━━━━━━ 33s 4ms/step - accuracy: 0.5540 -